# PROJETO DE DETECÇÃO DE FRAUDES EM COMPRAS ONLINE

FONTE: https://www.kaggle.com/datasets/ealaxi/paysim1?resource=download

IDEALIZADO EM: https://medium.com/coders-camp/230-machine-learning-projects-with-python-5d0c7abf8265

## IMPORT DE BIBLIOTECAS

### conectar com o google drive

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### integrar spark no colab
https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/

In [59]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null # download Java


In [60]:
# !wget -q https://www-us.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.7.tgz #  install Apache Spark 3.0.1 with Hadoop 2.7


In [61]:
# !tar xf spark-3.3.1-bin-hadoop3.tgz # unzip that folder


In [62]:
# !pip install -q findspark # install findspark library

### iniciar ambiente spark

In [63]:
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [64]:
# import findspark
# findspark.init()

In [65]:
# findspark.find()


In [66]:
# # import SparkSession from pyspark.sql and create a SparkSession, which is the entry point to Spark.
# from pyspark.sql import SparkSession

# spark = SparkSession.builder\
#         .master("local")\
#         .appName("Colab")\
#         .config('spark.ui.port', '4050')\
#         .getOrCreate()

In [67]:
# spark


### imports gerais

In [68]:
import pandas as pd
import numpy as np

## IMPORT DE FUNÇÕES

## IMPORT DE DADOS

In [69]:
# df = spark.read.csv("/content/drive/MyDrive/..Python_Codes/Projeto_fraude_compras_online/content/PS_20174392719_1491204439457_log.csv", header=True, inferSchema=True)
df = pd.read_csv("/content/drive/MyDrive/..Python_Codes/Projeto_fraude_compras_online/content/PS_20174392719_1491204439457_log.csv", header=0)  


In [70]:
print(f"Esse dataframe tem {len(df)} linhas e {len(df.columns)} colunas")
display(df)

Esse dataframe tem 6362620 linhas e 11 colunas


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


## TRATAMENTO E LIMPEZA DE DADOS

In [71]:
df_clean = df.dropna()

# One Hot Encoding e mapping para substituir string por número (necessário para ML)
df_clean['type_copy'] = df_clean['type']
df_clean = pd.get_dummies(df_clean, columns=['type'])
df_clean = df_clean.rename(columns={"type_copy": "type"})

# mapping
df_clean["type_num"] = df_clean["type"].map({"CASH_OUT": 1, 
                                       "PAYMENT": 2, 
                                        "CASH_IN": 3, 
                                        "TRANSFER": 4,
                                        "DEBIT": 5})

df_clean["isFraud_num"] = df_clean["isFraud"].map({0: "No Fraud", 1: "Fraud"})

# Indicador de quanto removeu dos dados
print(f"""
Sobraram neste dataframe {len(df_clean)} linhas e {len(df_clean.columns)} colunas
Equivalente à {round(100*(len(df_clean)/len(df)),2)} % das linas originais e {round(100*(len(df_clean.columns)/len(df.columns)),2)} % das colunas originais
""")

display(df_clean)


Sobraram neste dataframe 6362620 linhas e 18 colunas
Equivalente à 100.0 % das linas originais e 163.64 % das colunas originais



,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,type_num,isFraud_num
0,1,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0,PAYMENT,0,0,0,1,0,2,No Fraud
1,1,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0,PAYMENT,0,0,0,1,0,2,No Fraud
2,1,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0,TRANSFER,0,0,0,0,1,4,Fraud
3,1,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0,CASH_OUT,0,1,0,0,0,1,Fraud
4,1,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0,PAYMENT,0,0,0,1,0,2,No Fraud
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0,CASH_OUT,0,1,0,0,0,1,Fraud
6362616,743,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0,TRANSFER,0,0,0,0,1,4,Fraud
6362617,743,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0,CASH_OUT,0,1,0,0,0,1,Fraud
6362618,743,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0,TRANSFER,0,0,0,0,1,4,Fraud


## ANÁLISE

In [73]:
df_info = df_clean.copy()

# pie chart
transaction_type = df_info["type"].value_counts()
transactions = transaction_type.index
quantity = transaction_type.values

import plotly.express as px
figure = px.pie(df_info, 
             values=quantity, 
             names=transactions,hole = 0.5, 
             title="Distribuição de tipos de transação")
figure.show()

# Checking correlation
correlation = df_info.corr()
print('\n\n\nCorrelação entre variáveis:\n')
print(correlation["isFraud"].sort_values(ascending=False))

# Sklearn ML prediction

# splitting the data
from sklearn.model_selection import train_test_split
x = np.array(df_info[["type_num", "amount", "oldbalanceOrg", "newbalanceOrig"]]) # variaveis independentes
y = np.array(df_info[["isFraud_num"]]) # variaveis dependentes

# training a machine learning model
from sklearn.tree import DecisionTreeClassifier
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.10, random_state=42)
model = DecisionTreeClassifier()
model.fit(xtrain, ytrain)
print('\n\n\nScore do modelo:\n')
print(model.score(xtest, ytest))

# prediction
#features = [type, amount, oldbalanceOrg, newbalanceOrig]
features = np.array([[4, 9000.60, 9000.60, 0.0]])
print('\nPredict para TRANSFER, amount $9000.60, oldbalanceOrg $9000.60, newbalanceOrig $0.00:\n')
print(model.predict(features))




Correlação entre variáveis:

isFraud           1.000000
amount            0.076688
type_TRANSFER     0.053869
isFlaggedFraud    0.044109
step              0.031578
type_num          0.016171
type_CASH_OUT     0.011256
oldbalanceOrg     0.010154
newbalanceDest    0.000535
type_DEBIT       -0.002911
oldbalanceDest   -0.005885
newbalanceOrig   -0.008148
type_CASH_IN     -0.019089
type_PAYMENT     -0.025697
Name: isFraud, dtype: float64



Score do modelo:

0.9997343861491021

Predict para TRANSFER, amount $9000.60, oldbalanceOrg $9000.60, newbalanceOrig $0.00:

['Fraud']


## PLOT DE RESULTADOS


## ATUAÇÃO COM OUTSIDERS